In [1]:
%load_ext autoreload
%autoreload 2
from awesome.run.awesome_config import AwesomeConfig
from awesome.run.awesome_runner import AwesomeRunner
from awesome.util.reflection import class_name
import os
import torch

from awesome.dataset.sisbosi_dataset import SISBOSIDataset, ConvexityDataset as SISBOSIConvexityDataset
from awesome.dataset.convexity_segmentation_dataset import ConvexitySegmentationDataset
from awesome.measures.awesome_loss import AwesomeLoss
from awesome.measures.regularizer_loss import RegularizerLoss
from awesome.model.convex_diffeomorphism_net import ConvexDiffeomorphismNet
from awesome.model.net import Net
import awesome
from awesome.util.path_tools import get_project_root_path
from awesome.util.logging import basic_config
import matplotlib.pyplot as plt

basic_config()

os.chdir(get_project_root_path()) # Beeing in the root directory of the project is important for the relative paths to work consistently

D:\projects\AWESOME\awesome\agent\torch_agent.py:19: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
# CNNet

from awesome.dataset.awesome_dataset import AwesomeDataset
from awesome.dataset.fbms_sequence_dataset import FBMSSequenceDataset
from awesome.dataset.sisbosi_dataset import SISBOSIDataset, ConvexityDataset as SISBOSIConvexityDataset
from awesome.measures.awesome_image_loss_joint import AwesomeImageLossJoint
from awesome.measures.awesome_image_loss import AwesomeImageLoss
from awesome.measures.gradient_penalty_loss import GradientPenaltyLoss
from awesome.measures.fbms_joint_loss import FBMSJointLoss
from awesome.measures.regularizer_loss import RegularizerLoss
from awesome.model.cnn_net import CNNNet
from awesome.measures.tv import TV
from awesome.model.convex_net import ConvexNet
from awesome.model.unet import UNet
from awesome.measures.weighted_loss import WeightedLoss

xytype = "edgexy"
dataset_kind = "train"
dataset = "bear01"
all_frames = True
subset = None # slice(0, 5)

data_path = f"./data/local_datasets/FBMS-59/{dataset_kind}/{dataset}"
cfg = AwesomeConfig(
        name_experiment=f"UNET+{dataset}+{xytype}+REFIT",
        dataset_type=class_name(AwesomeDataset),
        dataset_args={
            "dataset": FBMSSequenceDataset(
                    dataset_path=data_path,
                    weak_labels_dir = "weak_labels/labels_with_uncertainty_flownet2_based",
                    processed_weak_labels_dir = "weak_labels/labels_with_uncertainty_flownet2_based/processed",
                    confidence_dir= "weak_labels/labels_with_uncertainty_flownet2_based/",
                    do_weak_label_preprocessing=True,
                    do_uncertainty_label_flip=True,
                    all_frames=all_frames
                ),
            "xytype": xytype,
            "feature_dir": f"{data_path}/Feat",
            "dimension": "3d", # 2d for fcnet
            "mode": "model_input",
            "model_input_requires_grad": False,
            "batch_size": 1,
            "split_ratio": 1,
            "shuffle_in_dataloader": True,
            "image_channel_format": "bgr",
            "do_image_blurring": True,
            "subset": subset
        },
        segmentation_model_type=class_name(UNet),
        segmentation_model_args={
            'in_chn': 6,
        },
        segmentation_training_mode='multi',
        #segmentation_model_state_dict_path=f"./data/checkpoints/retrain/model_{dataset}_unet.pth", # Path to the pretrained model
        use_segmentation_output_inversion=True,
        use_prior_model=False,
        loss_type=class_name(WeightedLoss),
        loss_args={
            "criterion": torch.nn.BCELoss(),
            "mode": "sssdms",
            "noneclass": 2,
        },
        use_step_lr_scheduler=True,
        step_lr_scheduler_args={
            "gamma": 0.1,
            "step_size": 5,
        },
        use_binary_classification=True, 
        num_epochs=15,
        device="cuda",
        dtype=str(torch.float32),
        runs_path="./runs/fbms_local/refit/",
        optimizer_args={
            "lr": 0.01,
            "betas": (0.9, 0.999),
            "eps": 1e-08,
            "amsgrad": False
        },
        use_progress_bar=True,
        semantic_soft_segmentation_code_dir="../siggraph/",
        semantic_soft_segmentation_model_checkpoint_dir="./data/sss_checkpoint/model",
        plot_indices_during_training_nth_epoch=5,
        compute_metrics_during_training_nth_epoch=5,
        plot_indices_during_training=[0, 19, 39, 59, 79, 99],
        agent_args=dict(
             do_pretraining=False,
        ),
        weight_decay_on_weight_norm_modules=0,
        #output_folder="./runs/fbms_local/unet/TestUnet/",
    )
cfg.save_to_file(f"./config/{cfg.name_experiment}.yaml", override=True, no_uuid=True)

Loading frames...:   0%|          | 0/100 [00:00<?, ?it/s]

'./config/UNET+bear01+edgexy+REFIT.yaml'

In [8]:
runner = AwesomeRunner(cfg)
runner.build()
runner.store_config()

Processing images...:   0%|          | 0/100 [00:00<?, ?it/s]

2023-11-12:17:51:07.271 INFO     [tensorboard.py:55] Tensorboard logger created at: runs\fbms_local\refit\UNET+bear01+edgexy+REFIT_23_11_12_17_51_07


'./runs/fbms_local/refit/UNET+bear01+edgexy+REFIT_23_11_12_17_51_07\\init_cfg_awesome_config.yaml'

In [9]:

runner.train()

  0%|          | 0/15 [00:00<?, ?it/s]

Computing metrics...:   0%|          | 0/100 [00:00<?, ?it/s]

Model-Epoch: 0 / LearningMode.TRAINING - Batches:   0%|          | 0/100 [00:00<?, ?it/s]

2023-11-12:17:52:26.580 INFO     [torch_agent.py:327] Epoch 1 / 15 time 00:00:31
2023-11-12:17:52:54.854 INFO     [torch_agent.py:327] Epoch 2 / 15 time 00:00:28
2023-11-12:17:53:24.110 INFO     [torch_agent.py:327] Epoch 3 / 15 time 00:00:29
2023-11-12:17:53:52.002 INFO     [torch_agent.py:327] Epoch 4 / 15 time 00:00:27


Computing metrics...:   0%|          | 0/100 [00:00<?, ?it/s]

...

Computing metrics...:   0%|          | 0/100 [00:00<?, ?it/s]

2023-11-12:17:58:39.974 INFO     [torch_agent.py:327] Epoch 10 / 15 time 00:01:24
2023-11-12:17:59:06.519 INFO     [torch_agent.py:327] Epoch 11 / 15 time 00:00:26
2023-11-12:17:59:33.005 INFO     [torch_agent.py:327] Epoch 12 / 15 time 00:00:26
2023-11-12:17:59:59.424 INFO     [torch_agent.py:327] Epoch 13 / 15 time 00:00:26
2023-11-12:18:00:26.017 INFO     [torch_agent.py:327] Epoch 14 / 15 time 00:00:26


Computing metrics...:   0%|          | 0/100 [00:00<?, ?it/s]

2023-11-12:18:01:41.620 INFO     [torch_agent.py:327] Epoch 15 / 15 time 00:01:15
2023-11-12:18:01:41.832 INFO     [handles.py:211] Saved checkpoint to ./runs/fbms_local/refit/UNET+bear01+edgexy+REFIT_23_11_12_17_51_07\checkpoint_epoch_15.pth at epoch 15
2023-11-12:18:01:41.834 INFO     [torch_agent.py:748] Training of agent UNET+bear01+edgexy+REFIT_23_11_12_17_51_07 complete in 00 days 00:09:46
2023-11-12:18:01:41.835 INFO     [torch_agent.py:752] Best model: UNET+bear01+edgexy+REFIT_Epoch_-1 Accuracy: 0.009373056713957339 Tag: train/epoch/WeightedLoss


Creating final images...:   0%|          | 0/100 [00:00<?, ?it/s]